In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [33]:
from sklearn.datasets import make_classification

X,y = make_classification(n_samples=10000, n_features=10, n_clusters_per_class=2, random_state=42)

In [34]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
print(type(X_train), type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(type(X_train), type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [38]:
# Removing correleated features

def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(pd.DataFrame(X_train), 0.8)

print('correlated features: ', len(set(corr_features)))

X_train = pd.DataFrame(X_train)

X_train.drop(labels=corr_features, axis=1, inplace=True)

X_test = pd.DataFrame(X_test)

X_test.drop(labels=corr_features, axis=1, inplace=True)



correlated features:  2


In [39]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='ovr')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)



/Users/mac/Desktop/MLOps/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [40]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold()

In [41]:
# Hiperparameter tuning

from sklearn.model_selection import GridSearchCV

penalty = ['l1', 'l2', 'elasticnet']
c_values = [100, 10, 1.0, 0.1, 0.01]
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

params = dict(penalty=penalty, C=c_values, solver=solver)

grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)

In [42]:
grid_search.fit(X_train, y_train)



/Users/mac/Desktop/MLOps/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/mac/Desktop/MLOps/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/mac/Desktop/MLOps/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/mac/Desktop/MLOps/venv/lib/python3.12/site-packages/sk

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=0, estimator=LogisticRegression(multi_class='ovr'),
             n_jobs=-1,
             param_grid={'C': [100, 10, 1.0, 0.1, 0.01],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [43]:
# Getting best parameters

print(grid_search.best_params_)

# Getting best score

print(grid_search.best_score_)

{'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
0.891375


In [44]:
y_pred = grid_search.predict(X_test)

In [45]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print(f"Model accuracy: {accuracy_score(y_test, y_pred)}")

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

Model accuracy: 0.8945
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       997
           1       0.90      0.89      0.89      1003

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000

[[894 103]
 [108 895]]


In [46]:
from sklearn.metrics import accuracy_score

print(f"Model accuracy: {accuracy_score(y_test, y_pred)}")

Model accuracy: 0.8945
